In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(g_key)

In [3]:
path=pd.read_csv("CityData.csv")
city_df=pd.DataFrame(path)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cabedelo,-6.98,-34.83,82.40,78,20,6.93,BR,1584485214
1,hobart,-42.88,147.33,71.01,49,100,10.29,AU,1584485214
2,wuchi,29.38,103.02,37.33,72,52,2.26,CN,1584485215
3,sept-iles,50.20,-66.38,27.00,92,90,16.11,CA,1584485120
4,qaanaaq,77.48,-69.36,-19.86,59,98,6.06,GL,1584485215


Humidity Heatmap
-Configure gmaps.
-Use the Lat and Lng as locations and Humidity as the weight.
-Add Heatmap layer to map.

In [4]:
city_df['Coords'] = list(zip(city_df["Lat"], city_df["Lng"]))
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Coords
0,cabedelo,-6.98,-34.83,82.40,78,20,6.93,BR,1584485214,"(-6.98, -34.83)"
1,hobart,-42.88,147.33,71.01,49,100,10.29,AU,1584485214,"(-42.88, 147.33)"
2,wuchi,29.38,103.02,37.33,72,52,2.26,CN,1584485215,"(29.38, 103.02)"
3,sept-iles,50.20,-66.38,27.00,92,90,16.11,CA,1584485120,"(50.2, -66.38)"
4,qaanaaq,77.48,-69.36,-19.86,59,98,6.06,GL,1584485215,"(77.48, -69.36)"


In [5]:
locations=city_df["Coords"]
weights=city_df["Humidity"]

In [9]:
# Plot Heatmap
equator_coordinates = (0.00, 0.00)
fig = gmaps.figure(center=equator_coordinates, zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=10.0))
fig

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
narrow_df=city_df.loc[(city_df["Max Temp"]<80) & (city_df["Max Temp"]>70) & \
                      (city_df["Wind Speed"]<10) & (city_df["Cloudiness"]==0)].dropna()
narrow_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Coords
214,mattru,7.62,-11.83,79.48,83,0,3.74,SL,1584485410,"(7.62, -11.83)"
231,airai,-8.93,125.41,72.90,67,0,4.52,TL,1584485416,"(-8.93, 125.41)"
266,yulara,-25.24,130.99,77.00,31,0,8.05,AU,1584485427,"(-25.24, 130.99)"
275,kovur,14.48,79.98,71.80,92,0,7.87,IN,1584485432,"(14.48, 79.98)"
290,sompeta,18.93,84.60,74.07,56,0,2.98,IN,1584485437,"(18.93, 84.6)"


In [11]:
hotel_df=narrow_df[["City", "Country", "Lat", "Lng", "Coords"]]

In [12]:
hotel_df["Hotel Name"]= " "
hotel_df.count()

C:\Users\mjensen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


City          17
Country       17
Lat           17
Lng           17
Coords        17
Hotel Name    17
dtype: int64

In [13]:
hotel_df.head()

,City,Country,Lat,Lng,Coords,Hotel Name
214,mattru,SL,7.62,-11.83,"(7.62, -11.83)",
231,airai,TL,-8.93,125.41,"(-8.93, 125.41)",
266,yulara,AU,-25.24,130.99,"(-25.24, 130.99)",
275,kovur,IN,14.48,79.98,"(14.48, 79.98)",
290,sompeta,IN,18.93,84.60,"(18.93, 84.6)",


In [14]:
params={
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
    "keyword": "hotel"
}

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]= str(lat)+","+str(lng)
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(base_url, params=params).json()
#     print(response.url)
    results=response['results']
    try:
        hotel_df.loc[index, "Hotel Name"]=results[0]["name"]
    except(KeyError, IndexError):
        print("Missing Field")

hotel_df

C:\Users\mjensen\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing Field
Missing Field


,City,Country,Lat,Lng,Coords,Hotel Name
214,mattru,SL,7.62,-11.83,"(7.62, -11.83)",Bo inn Hotel
231,airai,TL,-8.93,125.41,"(-8.93, 125.41)",Restaurant Gabmenis
266,yulara,AU,-25.24,130.99,"(-25.24, 130.99)",Ayers Rock Campground
275,kovur,IN,14.48,79.98,"(14.48, 79.98)",Sri Vinayaka Lodge
290,sompeta,IN,18.93,84.60,"(18.93, 84.6)",Haritha Beach Resort
312,chandbali,IN,20.78,86.77,"(20.78, 86.77)",Hussain Guest House
356,oussouye,SN,12.48,-16.55,"(12.48, -16.55)",Hotel Carabane
364,keti bandar,PK,24.14,67.45,"(24.14, 67.45)",
370,bera,BD,24.08,89.62,"(24.08, 89.62)",Shahin Hotel
377,shwebo,MM,22.57,95.70,"(22.57, 95.7)",MOON SHINE Guest House


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))